In [1]:
import ee
ee.Initialize()

In [2]:
from iteru import *

In [3]:
import tempfile 
import requests
import os
import matplotlib.pyplot as plt

In [4]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [5]:
startYear = 2020
startMonth =6
startDay = 1
endYear = 2022
endMonth = 3
endDay = 1

In [6]:
SAR,dates_sequences = S1_SAR_col(GERD_aoi,startYear,startMonth,startDay,endYear,endMonth,endDay)

In [7]:
# url = SAR_timeseries_url(SAR,GERD_aoi)

In [8]:
# out_gif = get_gif(url)
# add_text_to_gif(out_gif,dates_sequences, dates_font_size = 25, dates_font_color = 'white', framesPerSecond = 2)
# display_gif(out_gif)

In [9]:
# water_vectors = SAR.map(filterSpeckles).map(water_classify).map(water_to_vector)

In [10]:
#water_area = [area /(1e6) for area in water_vectors.aggregate_array('Area').getInfo()]

In [11]:
# dems = water_vectors.map(max_water_ele)

In [17]:
# max_elev = [elev for elev in dems.aggregate_array('Maximum_water_elevation').getInfo()]

In [18]:
# volume_stats = dems.map(water_vol)

In [19]:
# ele_sum = volume_stats.aggregate_array('Elevation_sum').getInfo()

In [20]:
# pixel_num  = volume_stats.aggregate_array('Pixels_number').getInfo()

In [21]:
# volume = [((water_level*pixles_count-elevations_sum)*900)/(1e9) for water_level,elevations_sum , pixles_count in zip(max_elev,ele_sum,pixel_num)]